In [1]:
import tensorflow as tf

import numpy as np
import time

In [2]:
def HVP(y, x, v):
    # y: scalar loss function
    # w: weight vector in order to generate Hessian
    # x: feed_dict value for input of the network
    # v: vector to be producted (gradient dimension, i.e. list)
    w = tf.trainable_variables(scope='network')
    
    grads = tf.gradients(y, w)
    
    grads_inner = grad_inner_product(grads, v)
    
    hvp = tf.gradients(grads_inner, w)
    
    hvp_val = list(map(lambda k: k.eval(x),hvp))
    
    return hvp_val

In [3]:
def grad_inner_product(grad1, grad2):
    # inner product for list-format gradients (output scalar value)
    
    val = 0
    
    for i in range(len(grad1)):
        val += np.sum(np.multiply(grad1[i], grad2[i]))
        
    return val

In [4]:
# toy example for HVP

with tf.variable_scope('network'):
    x = tf.placeholder(tf.float32, (None, 1))
    h = tf.layers.dense(x, 1, activation=None, use_bias=False)
    y = tf.layers.dense(h, 1, activation=None, use_bias=False)

In [5]:
vars = tf.trainable_variables(scope='network')

tf.InteractiveSession()
tf.global_variables_initializer().run()

x_feed = {x: [[1.]]}
v_feed = [np.ones_like(p) for p in vars] # gradient_like one vector
print(v_feed)

hvp = HVP(y, x_feed, v_feed)
print('original hvp:', hvp)
print('concated hvp:', np.concatenate(hvp))

np.concatenate(hvp).shape

[array(1, dtype=object), array(1, dtype=object)]
original hvp: [array([[1.]], dtype=float32), array([[1.]], dtype=float32)]
concated hvp: [[1.]
 [1.]]


(2, 1)

hvp를 할 때 훨씬 정확한 값이 나옴.

차이가 뭐지?

automatic differentiation을 사용하기 때문이라고 함.
refer: https://en.wikipedia.org/wiki/Automatic_differentiation

------------

cf) 
automatic differentiation은 
- symboloic differentiation 도 아니고,
    - mathematica에서 사용하는 방법. sine, cosine 등의 미분식을 이용해서 미분식을 얻음.
- numerical differentiation 도 아님.
    - (f(x+h)-f(x))/h를 사용해서 미분값을 찾음.
    
refer: https://www.youtube.com/watch?v=pdSCHtPJ4B0

approximation이 아니라 정확한 differetiate value를 얻을 수 있음.

원 함수 f를 primal이라고 함.
이 함수의 미분 f'을 dual이라고 함.

'all algorithms are compositions of a finite set of elementary operations (with known derivatives)'

e.g. 
f(a,b):
  c = a * b
  d = sin c
  return d
  
f'(a,a',b,b'):
  (c,c') = (a*b, a'*b + a*b')
  (d,d') = (sinc, c' * cosc) <- c에 대한 미분
  return (d,d')

각 layer마다 derivative를 주고, (예를 들자면 relu 같은 경우 명시적으로 derivative를 줌)
layer가 쌓이는 것은 chain rule을 사용하면, (또 필요에 의하면 기본 calculus rule (e.g. (a+b)' = a'+b')을 사용해서 전체 closed form derivative를 얻을 수 있음.

Q: 그러면 symbolic differentiation인거 아닌가?

AD has two main modes:

- forward mode
- reverse mode

e.g.
## forward mode

$y = f(x1, x2) = ln(x_1) + x1x2 - sin(x2)$

간단해보이는 한 줄 식이지만, 사실 이 값을 계산하려면 computational graph가 필요함. 
(low level 단에서 생각)

![computational_graph](./computational_graph.png)

- v-1: x1
- v0: x2
- v1: ln(x1) <=> ln(v-1)
- v2: x1x2 <=> v-1v0
- v3: sin(x2) <=> sin(v0)
- v4: ln(x1) + x1x2 <=> v1 + v2
- v5: ln(x1) + x1x2 - sin(x2) <=> v3 + v4

이렇게 operation 단위로 쪼개지게 되면 calculus rule, chain rule을 쉽게 적용할 수 있고, 이 low level 단에서 미분을 진행시켜주는게 AD (automatic differentiate).

cf) 각각의 chain rule을 써서 미분을 구할 때 symbolic이 아니라 __미분 값__ 자체를 얻은 후 진행됨.
(그렇기 때문에 (구현을 쉽게 하기 위해서) cntk에서는 grad를 operator로 만들지 않고, value based로 한 것인듯.)

![forward_trace](./forward_trace.png)

오른쪽 forward derivative trace는 x1에 대해서 미분을 구하는 것.
보면 알 수 있겠지만, 각 primary operation의 미분식은 주어져야 함.
~~이 경우는 매우 단순해서 chain rule 마저 필요없이 전단에서 구한 값을 그대로 대입하면 됨.~~

## reverse mode

backpropagation is just a special case of reverse mode AD

~~앞에서는 x1,x2로 end에서 시작해서 output도 x1,x2의 함수인 것으로 매우 단순한 경우였었는데, neural network처럼 중간중간 w1, ..., wk가 등장해 chain rule을 사용해야 하는 경우 chain rule까지 써서 모든 gradient 값을 찾겠다는 것.~~

reverse mode는 y1에 대해서 미분을 구하는 것. 마찬가지로 각 primary operation의 미분식은 주어져야 함.

example을 복잡성이 문제가 아니라 계산의 효율성에 관한 것임. 밑 forward mode v.s. reverse mode 참고.

![reverse_trace](./reverse_trace.png)

cf) $\bar{v}_i=\frac{\partial y}{\partial v_i}$

x1을 비교해보면 5.5로 서로 값이 같다는 걸 확인할 수 있음.

(reverse mode가 필요한 이유가 뭐지? -> x1, x2에 대해서 한 번에 구할 수 있음.)

## forward mode v.s. reverse mode

- forward mode: y가 vector고 x가 scalar인 경우, x에 대해서 한 번에 y1, ..., yk의 미분값을 얻을 수 있음.
- reverse mode: y가 scalar고 x가 vector인 경우, y에 대해서 한 번에 x1, ..., xn의 미분값을 얻을 수 있음.

f:R^n -> R^m의 경우, jacobian J \in R^{mxn}이

- O(n x time(f)) with forward AD
- O(m x time(f)) with reverse AD

걸림. 따라서 m,n을 비교하면서 유용한 것을 택하면 계산을 효율적으로 할 수 있음.